In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/Pattern\ Lab/Datasets/Datasets\ SL/classified_datasets
path = "/content/drive/My\ Drive/Pattern\ Lab/Datasets/Datasets\ SL/classified_datasets"

/content/drive/My Drive/Pattern Lab/Datasets/Datasets SL/classified_datasets


In [ ]:
# all library imports here
import pandas as pd
import numpy as np
import os
import pickle

In [ ]:
a_file = open("index_dictionary.pkl", "rb")
index_dictionary = pickle.load(a_file)
print(index_dictionary)

{'binary': [0, 163], 'CKSNAP1': [164, 195], 'CKSNAP3': [196, 259], 'CKSNAP5': [260, 355], 'CKSNAP7': [356, 483], 'CKSNAP9': [484, 643], 'ENAC5': [644, 791], 'ENAC10': [792, 939], 'DAC': [940, 981], '_EIIP': [982, 1022], 'PseEIIP': [1023, 1086], 'NCP': [1087, 1209], 'Kmer1': [1210, 1213], 'Kmer2': [1214, 1229], 'Kmer3': [1230, 1293], 'Kmer4': [1294, 1549], 'Kmer5': [1550, 2573], 'TAC': [2574, 2587]}


In [ ]:
import random

def getList(dict): 
    return list(dict.keys()) 

       
keyList = getList(index_dictionary)
print(type(keyList))

random_feature = [] 

for i in range(4):
  random_feature.append(random.sample(keyList, 5))

print(random_feature )

<class 'list'>
[['_EIIP', 'Kmer4', 'NCP', 'CKSNAP7', 'ENAC5'], ['DAC', 'ENAC10', 'ENAC5', 'Kmer1', 'CKSNAP3'], ['_EIIP', 'CKSNAP9', 'DAC', 'PseEIIP', 'ENAC5'], ['Kmer1', 'Kmer4', 'Kmer2', 'PseEIIP', 'Kmer3']]


In [ ]:
Xa_train, Xa_test, ya_train, ya_test = np.load("xa_train.npy"), np.load("xa_test.npy"), np.load("ya_train.npy"), np.load("ya_test.npy")
Xc_train, Xc_test, yc_train, yc_test = np.load("xc_train.npy"), np.load("xc_test.npy"), np.load("yc_train.npy"), np.load("yc_test.npy")
Xd_train, Xd_test, yd_train, yd_test = np.load("xd_train.npy"), np.load("xd_test.npy"), np.load("yd_train.npy"), np.load("yd_test.npy")
print(Xa_train.shape, Xa_test.shape, ya_train.shape, ya_test.shape)
print(Xc_train.shape, Xc_test.shape, yc_train.shape, yc_test.shape)
print(Xd_train.shape, Xd_test.shape, yd_train.shape, yd_test.shape)

(3690, 2568) (262, 2568) (3690, 1) (262, 1)
(2898, 2568) (206, 2568) (2898, 1) (206, 1)
(3300, 2568) (234, 2568) (3300, 1) (234, 1)


#pa_test = np.concatenate((pa_test, dt), axis=1)
for feature_groups in random_feature:
  #feature_groups = random_feature[0] 
  no_of_features = 0
  for i in feature_groups:
    no_of_features = no_of_features + index_dictionary[i][1]-index_dictionary[i][0]+1
  print(no_of_features)

  xa_train_n, xa_test_n, ya_train_n, ya_test_n = np.zeros((Xa_train.shape[0],no_of_features)), np.zeros((Xa_test.shape[0],no_of_features)), ya_train, ya_test
  starting_index = 0
  for i in feature_groups:
    ending_index = starting_index + index_dictionary[i][1]-index_dictionary[i][0]
    print(starting_index,ending_index)
    print(index_dictionary[i][1]+1, index_dictionary[i][0])
    xa_train_n[:,starting_index:ending_index+1] = Xa_train[:,index_dictionary[i][0]:index_dictionary[i][1]+1]
    xa_test_n[:,starting_index:ending_index+1] = Xa_test[:,index_dictionary[i][0]:index_dictionary[i][1]+1]
    starting_index = ending_index+1
  print(starting_index)

In [ ]:
#pa_test = np.concatenate((pa_test, dt), axis=1)
xa_trains = []
for i, feature_groups in enumerate(random_feature):
  #no_of_features = 0
  #for j in feature_groups:
   # no_of_features = no_of_features + index_dictionary[j][1]-index_dictionary[j][0]+1
  #print(no_of_features)
  xa_train_temp = np.concatenate([Xa_train[ : ,  index_dictionary[x][0] : index_dictionary[x][1]+1] for x in feature_groups], axis = 1)
  xa_trains.append(xa_train_temp)
  xa_test_temp = np.concatenate([Xa_test[ : ,  index_dictionary[x][0] : index_dictionary[x][1]+1] for x in feature_groups], axis = 1)
  xa_tests.append(xa_test_temp)
  print(xa_trains[i].shape)

(3690, 696)
(3690, 406)
(3690, 455)
(3690, 404)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
import pickle
import random

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, matthews_corrcoef
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from tabulate import tabulate
from IPython.display import clear_output

In [ ]:
def avg(avg_list):
  return sum(avg_list)/len(avg_list)

In [ ]:
# for SVM
cv = StratifiedKFold( n_splits=5, shuffle=True)
svc = SVC() # Support vector classifier(SVC)

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []

svc_rows = []
headers = ["Random-Set", "Classifier", "Train-Accuracy", "Test-Accuracy", "Train-Specificity", "Test-Specificity", "Train-Sensitivity", "Test-Sensitivity", "Train-MCC", "Test-MCC"]

for i, x in enumerate(xa_trains):
  for train_index, test_index in cv.split(x, ya_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = ya_train[train_index], ya_train[test_index]
      svc.fit(x_train, y_train.ravel())
      test_pred = svc.predict(x_test)
      train_pred = svc.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  svc_rows.append(["Set0"+str(i+1), "SVM", avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])
clear_output()
print("For SVM Classifier:\n")
table = tabulate(svc_rows, headers, tablefmt='simple')
print(table)

 For SVM Classifier:

Random-Set    Classifier      Train-Accuracy    Test-Accuracy    Train-Specificity    Test-Specificity    Train-Sensitivity    Test-Sensitivity    Train-MCC    Test-MCC
------------  ------------  ----------------  ---------------  -------------------  ------------------  -------------------  ------------------  -----------  ----------
Set01         SVM                   0.869919         0.77019              0.875745            0.77019              0.87429             0.770323     0.739894    0.540518
Set02         SVM                   0.859756         0.757859             0.870461            0.762873             0.867569            0.76063      0.719719    0.515978
Set03         SVM                   0.855059         0.751581             0.866576            0.757724             0.863382            0.7549       0.710354    0.503512
Set04         SVM                   0.809299         0.707724             0.830047            0.722629             0.820203          

In [ ]:
###  for LogisticRegression

logisticRegression = LogisticRegression(max_iter=500)
#cv = StratifiedKFold( n_splits=5, shuffle=True)

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []

lg_rows = []

for i, x in enumerate(xa_trains):
  for train_index, test_index in cv.split(x, ya_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = ya_train[train_index], ya_train[test_index]
      logisticRegression.fit(x_train, y_train.ravel())
      test_pred = logisticRegression.predict(x_test)
      train_pred = logisticRegression.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  lg_rows.append(["Set0"+str(i+1), "LogisticRegression", avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),

                  avg(train_mcc), avg(test_mcc)])
  
clear_output()
print("For LogisticRegression:\n")
table = tabulate(lg_rows, headers, tablefmt='simple')
print(table)

 For LogisticRegression

Random-Set    Classifier            Train-Accuracy    Test-Accuracy    Train-Specificity    Test-Specificity    Train-Sensitivity    Test-Sensitivity    Train-MCC    Test-MCC
------------  ------------------  ----------------  ---------------  -------------------  ------------------  -------------------  ------------------  -----------  ----------
Set01         LogisticRegression          0.805894         0.77019              0.802846            0.768564             0.804059            0.769666     0.611804    0.540814
Set02         LogisticRegression          0.798713         0.767344             0.799255            0.765583             0.799015            0.766798     0.597444    0.535087
Set03         LogisticRegression          0.797945         0.764408             0.798735            0.764408             0.798401            0.764614     0.595904    0.529107
Set04         LogisticRegression          0.771443         0.73977              0.774255            

In [ ]:
###  for XGBClassifier

xgbC = XGBClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
xgbc_rows = []
for i, x in enumerate(xa_trains):
  for train_index, test_index in cv.split(x, ya_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = ya_train[train_index], ya_train[test_index]
      xgbC.fit(x_train, y_train.ravel())
      test_pred = xgbC.predict(x_test)
      train_pred = xgbC.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  xgbc_rows.append(["Set0"+str(i+1), "XGB", avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])

clear_output()
print("For XGBClassifier:\n")
table = tabulate(xgbc_rows, headers, tablefmt='simple')
print(table)

For XGBClassifier

Random-Set    Classifier      Train-Accuracy    Test-Accuracy    Train-Specificity    Test-Specificity    Train-Sensitivity    Test-Sensitivity    Train-MCC    Test-MCC
------------  ------------  ----------------  ---------------  -------------------  ------------------  -------------------  ------------------  -----------  ----------
Set01         XGB                   0.864702         0.771816             0.873848            0.773442             0.871513            0.772824     0.729541    0.544111
Set02         XGB                   0.85481          0.750271             0.863482            0.752033             0.861124            0.751153     0.709756    0.500871
Set03         XGB                   0.857972         0.755736             0.865763            0.753568             0.863664            0.754601     0.716054    0.511754
Set04         XGB                   0.839245         0.740515             0.85227             0.744715             0.847838            0

In [ ]:
###  for AdaBoostClassifier

adaBC = AdaBoostClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
adabc_rows = []

for i, x in enumerate(xa_trains):
  for train_index, test_index in cv.split(x, ya_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = ya_train[train_index], ya_train[test_index]
      adaBC.fit(x_train, y_train.ravel())
      test_pred = adaBC.predict(x_test)
      train_pred = adaBC.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  adabc_rows.append(["Set0"+str(i+1),"AdaBoost", avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])

clear_output()
print("For AdaBoostClassifier:\n")
table = tabulate(adabc_rows, headers, tablefmt='simple')
print(table)

For AdaBoostClassifier

Random-Set    Classifier      Train-Accuracy    Test-Accuracy    Train-Specificity    Test-Specificity    Train-Sensitivity    Test-Sensitivity    Train-MCC    Test-MCC
------------  ------------  ----------------  ---------------  -------------------  ------------------  -------------------  ------------------  -----------  ----------
Set01         AdaBoost              0.791057         0.749322             0.78645             0.752846             0.788451            0.751758     0.582165    0.49901
Set02         AdaBoost              0.774627         0.727913             0.773171            0.730352             0.773772            0.729535     0.549289    0.45646
Set03         AdaBoost              0.778771         0.731978             0.777236            0.731707             0.777887            0.73215      0.557582    0.464476
Set04         AdaBoost              0.763567         0.717344             0.767717            0.724119             0.765321          

In [ ]:
###  for RandomForestClassifier

rfC = RandomForestClassifier()

train_accuracy=[]
train_specificity = []
train_sensitivity = []
train_mcc = []
test_accuracy=[]
test_specificity = []
test_sensitivity = []
test_mcc = []
rfc_rows = []

for i, x in enumerate(xa_trains):
  for train_index, test_index in cv.split(x, ya_train):
      x_train, x_test = x[train_index], x[test_index]
      y_train, y_test = ya_train[train_index], ya_train[test_index]
      rfC.fit(x_train, y_train.ravel())
      test_pred = rfC.predict(x_test)
      train_pred = rfC.predict(x_train)

      ## Accuracy
      train_accuracy.append(accuracy_score(y_train, train_pred))
      test_accuracy.append(accuracy_score(y_test, test_pred))

      ## confusion metrics
      tn, fp, fn, tp = confusion_matrix(y_train, train_pred).ravel()
      train_specificity.append(tn / (tn + fp)) # Specificity
      train_sensitivity.append(tp / (tp + fp)) # Sensitivity
      tn, fp, fn, tp = confusion_matrix(y_test, test_pred).ravel()
      test_specificity.append(tn / (tn + fp))
      test_sensitivity.append(tp / (tp + fp))

      ## MCC
      train_mcc.append(matthews_corrcoef(y_train, train_pred))
      test_mcc.append(matthews_corrcoef(y_test, test_pred))

  print("Random set -:", i+1)
  print("Train-Accuracy = ", avg(train_accuracy))
  print("Test-Accuracy = ",avg(test_accuracy))
  print("Train-Specificity = ", avg(train_specificity))
  print("Test-Specificity = ", avg(test_specificity))
  print("Train-Sensitivity = ", avg(train_sensitivity))
  print("Test-Sensitivity = ", avg(test_sensitivity))
  print("Train-MCC = ", avg(train_mcc))
  print("Test-MCC = ", avg(test_mcc))
  print("\n\n")
  rfc_rows.append(["Set0"+str(i+1),"RandomForest", avg(train_accuracy), avg(test_accuracy), avg(train_specificity), avg(test_specificity), avg(train_sensitivity), avg(test_sensitivity),
                  avg(train_mcc), avg(test_mcc)])

clear_output()
print("For RandomForestClassifier:\n")
table = tabulate(rfc_rows, headers, tablefmt='simple')
print(table)

For RandomForestClassifier

Random-Set    Classifier      Train-Accuracy    Test-Accuracy    Train-Specificity    Test-Specificity    Train-Sensitivity    Test-Sensitivity    Train-MCC    Test-MCC
------------  ------------  ----------------  ---------------  -------------------  ------------------  -------------------  ------------------  -----------  ----------
Set01         RandomForest                 1         0.734146                    1            0.733333                    1            0.734231            1    0.468597
Set02         RandomForest                 1         0.715312                    1            0.718157                    1            0.716771            1    0.430925
Set03         RandomForest                 1         0.716441                    1            0.719241                    1            0.717743            1    0.433167
Set04         RandomForest                 1         0.710298                    1            0.717073                    1    

In [ ]:
table = tabulate(svc_rows, headers, tablefmt='simple')
print(table,"\n")
table = tabulate(lg_rows, headers, tablefmt='simple')
print(table,"\n")
table = tabulate(xgbc_rows, headers, tablefmt='simple')
print(table,"\n")
table = tabulate(adabc_rows, headers, tablefmt='simple')
print(table,"\n")
table = tabulate(rfc_rows, headers, tablefmt='simple')
print(table,"\n")

Random-Set    Classifier      Train-Accuracy    Test-Accuracy    Train-Specificity    Test-Specificity    Train-Sensitivity    Test-Sensitivity    Train-MCC    Test-MCC
------------  ------------  ----------------  ---------------  -------------------  ------------------  -------------------  ------------------  -----------  ----------
Set01         SVM                   0.869919         0.77019              0.875745            0.77019              0.87429             0.770323     0.739894    0.540518
Set02         SVM                   0.859756         0.757859             0.870461            0.762873             0.867569            0.76063      0.719719    0.515978
Set03         SVM                   0.855059         0.751581             0.866576            0.757724             0.863382            0.7549       0.710354    0.503512
Set04         SVM                   0.809299         0.707724             0.830047            0.722629             0.820203            0.711988     0.61922